In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

## 데이터 준비

In [2]:
# -------------------------------------------------
# data 불러오기
# -------------------------------------------------
df = pd.read_csv('C:/Users/EL087/Desktop/MS_MachineLearning/data/bike_sharing_demand.csv', parse_dates=['datetime'])

# 전처리

### 파생컬럼 추가

In [3]:
# --------------------------------------------
# 연, 월, 시, 요일
# --------------------------------------------
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['hour'] = df['datetime'].dt.hour
df['dayofweek'] = df['datetime'].dt.dayofweek # 월요일:0, 일요일:6
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,hour,dayofweek
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,1,0,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,1,5
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,1,2,5
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011,1,3,5
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011,1,4,5


### 변수선택

In [4]:
# --------------------------------------------
# 독립변수, 종속변수선택
# --------------------------------------------

# 독립변수에 사용하지 않을 컬럼
del_cols = ['datetime','casual','registered','count','temp'] 

X = df.drop(del_cols, axis=1).copy()
y = df['count']

X.head()

,season,holiday,workingday,weather,atemp,humidity,windspeed,year,month,hour,dayofweek
0,1,0,0,1,14.395,81,0.0,2011,1,0,5
1,1,0,0,1,13.635,80,0.0,2011,1,1,5
2,1,0,0,1,13.635,80,0.0,2011,1,2,5
3,1,0,0,1,14.395,75,0.0,2011,1,3,5
4,1,0,0,1,14.395,75,0.0,2011,1,4,5


In [5]:
# ------------------------------------------------
# 독립변수 - 범주형, 수치형, 순환형 변수 구분
# ------------------------------------------------
cat_cols = ['season','holiday','workingday','weather']      
num_cols = ['atemp','humidity','windspeed', 'year']
cycle_cols = ['month', 'hour', 'dayofweek']

### 훈련세트/테스트세트 분할

In [6]:
# 훈련세트/테스트세트 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(8164, 11) (2722, 11) (8164,) (2722,)


### 전처리기 구성

* 여러 종류의 전처리 작업을 효율적이고 일관성 있게, 한 번에 처리하기 위해 사용

* ColumnTransformer(동시 처리)
    * 목적: 서로 다른 유형의 열(Columns)에 대해 다른 전처리(Scaling, Encoding 등)를 한 번에 적용
    * 여러 전처리기를 묶어서 하나의 변환기를 만드는 객체


In [7]:
# FunctionTransformer에 사용될 순환형 데이터 인코딩 함수 정의
def cyclical_feature_engineer(X, feature_names):
    """
    주어진 열에 대해 sin/cos 인코딩을 수행하고 원본 특성을 유지하여 Numpy 배열 형식으로 반환
    """
    # X는 ColumnTransformer로부터 NumPy 배열로 전달되므로, DataFrame으로 변환 후 처리합니다.
    X_df = pd.DataFrame(X, columns=feature_names) 
    
    X_new_list = []
    
    # 각 순환형 특성에 대해 인코딩 수행 (주기: hour=24, month=12, dayofweek=7)
    cycle_map = {'hour': 24, 'month': 12, 'dayofweek': 7}
    
    for feature in feature_names:
        cycle = cycle_map.get(feature)
        if cycle:
            # sin/cos 값 계산 후 리스트에 추가
            X_new_list.append(np.sin(2 * np.pi * X_df[feature] / cycle).values)
            X_new_list.append(np.cos(2 * np.pi * X_df[feature] / cycle).values)
            
    # 새로 생성된 특성들을 수평(axis=1)으로 합쳐 NumPy 배열로 반환
    return np.stack(X_new_list, axis=1)

In [8]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
# -----------------------------
# ColumnTransformer 구성
#   - 순환형 데이터 인코딩
#   - "cat" : 범주형 → OneHotEncoder
#   - "num" : 수치형 → StandardScaler
#   remainder="passthrough" : transformers에 포함되지 않은 컬럼은 변환 없이 그대로 유지
# -----------------------------
preprocessor = ColumnTransformer(
    transformers=[
        ("cyclical",
         FunctionTransformer(cyclical_feature_engineer, validate=False, kw_args={'feature_names': cycle_cols}),
         cycle_cols  # ['month', 'hour', 'dayofweek'] 열만 이 변환기에 전달)
        ),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols),
        ("num", StandardScaler(), num_cols)
    ],
    remainder="drop"   # cycle_cols에 사용된 원본 3개 열이 최종 출력에서 제거
)

### 파이프라인 구성
* 파이프라인 (순서 관리)

    * 목적: 전처리 단계들(변환기)과 최종 모델(추정기)을 순차적인 하나의 워크플로우로 연결하고 관리
    * 데이터 처리 과정을 순차적으로 연결하여 자동화하는 도구

In [9]:
from sklearn.pipeline import Pipeline # 파이프라인 클래스
from sklearn.ensemble import RandomForestRegressor # 모델 클래스

# --------------------------------------------
# 파이프라인 정의
# --------------------------------------------
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(random_state=42))
])

# --------------------------------------------
# 훈련
# --------------------------------------------
pipeline.fit(X_train, y_train)

# --------------------------------------------
# 예측
# --------------------------------------------
y_pred = pipeline.predict(X_test)

# --------------------------------------------
# 모델 성능 평가
# --------------------------------------------
from sklearn.metrics import r2_score, root_mean_squared_error, mean_absolute_error, mean_squared_error
r2 = r2_score(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f'r2:{r2}')
print(f'rmse:{rmse}')
print(f'mae:{mae}')
print(f'mse:{mse}')

# --------------------------------------------
# 과적합 확인
# --------------------------------------------
print('train>>>>>>>>>>>>', pipeline.score(X_train, y_train))
print('test>>>>>>>>>>>>>', pipeline.score(X_test, y_test))

r2:0.9527601607714882
rmse:39.37169006476704
mae:24.57303459466079
mse:1550.1299785560757
train>>>>>>>>>>>> 0.9922765080878488
test>>>>>>>>>>>>> 0.9527601607714882


### 파이프라인 다운로드


In [10]:
import joblib

# ----------------------
# 파이프라인 저장
# ----------------------
joblib.dump(pipeline, "C:/Users/EL087/Desktop/MS_MachineLearning/model/bike_rent_pipe.pkl")

['C:/Users/EL087/Desktop/MS_MachineLearning/model/bike_rent_pipe.pkl']

- .pkl 파일의 의미와 사용

    **1. 객체의 직렬화 (Serialization)**

- 의미: $\text{.pkl}$ 파일은 파이썬 프로그램에서 사용되는 **객체(예: 훈련된 머신러닝 모델, 전처리기, 복잡한 데이터 구조 등)**의 상태를 그대로 저장한 것

- 목적: 프로그램을 다시 실행할 때, 학습에 오래 걸리는 모델을 처음부터 다시 훈련할 필요 없이 저장된 $\text{.pkl}$ 파일을 불러와서(역직렬화) 즉시 예측에 사용할 수 있게 함

    **2. 머신러닝에서의 활용**

    머신러닝 프로젝트에서 $\text{.pkl}$은 다음과 같은 핵심 구성 요소를 저장하는 데 필수적으로 사용

- 훈련된 모델: $\text{RandomForestRegressor}$나 $\text{LinearRegression}$ 같은 훈련이 완료된 모델 객체를 저장하여 배포

- 전처리기: $\text{StandardScaler}$나 $\text{OneHotEncoder}$처럼 훈련 데이터의 통계 정보(평균, 범주 목록 등)를 학습한 전처리기 객체를 저장,   
이를 통해 새로운 테스트 데이터에도 동일한 기준으로 전처리를 적용할 수 있음

### 새로운 데이터 예측

In [11]:
import joblib

# ------------------
# 파이프라인 불러오기
# ------------------
file_path = "C:/Users/EL087/Desktop/MS_MachineLearning/model/bike_rent_pipe.pkl"
loaded_pipe = joblib.load(file_path)

In [12]:
# ------------------
# 예측 
# ------------------

new_row = {
    "season": 2, "holiday": 0, "workingday": 1, "weather": 3,
    "atemp": 20.5, "humidity": 55, "windspeed": 0.12,
    "year": 2025, "month": 5, "day": 1, "hour": 17, "dayofweek": 3
}

new_df = pd.DataFrame([new_row])

# Pipeline 내부에서 자동으로 → 순환형 데이터 인코딩 -> One‑Hot → 스케일링 → 예측
count_pred = loaded_pipe.predict(new_df)[0]
print(f"예상 대여 수: {count_pred:.0f}대")

예상 대여 수: 603대
